In [1]:
from fastai.vision import *

# DATA INSPECTION

In [ ]:
classes = os.listdir("data/mountains")[1:]
print(classes)

## View/Load data

In [2]:
path = 'data/mountains'

In [3]:
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, num_workers=4, bs=32).normalize(imagenet_stats)
# if we don't have a trn, val, test set => passing 'train="."' says that the current folder contains the trn data,
# and 'valid_pct=0.2' will set aside randomely 20% of the data.

In [ ]:
data.show_batch(rows=3, figsize=(7,8))

In [ ]:
print("classes: ", data.classes)
print("nb classes: ", data.c)
print("nb training examples: ", len(data.train_ds))
print("nb val examples", len(data.valid_ds))

# TRAINING

Create models folder and grant access if not already done

In [ ]:
#! sudo mkdir /home/jupyter/tutorials/fastai/course-v3/nbs/dl1/data/mountains/models
#! sudo chmod -R 777 /home/jupyter/tutorials/fastai/course-v3/nbs/dl1/data/mountains/models

Load a pre-trained imagenet RN-50 model and train on the mountains dataset.

In [4]:
learn = cnn_learner(data, models.resnet50, metrics=error_rate)

Train head for 5 epochs.

In [ ]:
learn.load("body-40-epochs");

In [5]:
learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,error_rate,time
0,3.668491,2.566968,0.781513,00:15
1,3.280624,2.554640,0.766106,00:13
2,3.091088,2.584032,0.746499,00:13
3,2.833698,2.365849,0.705882,00:13
4,2.566943,2.362857,0.704482,00:13
5,2.340815,2.303739,0.701681,00:13
6,2.240762,2.159841,0.676471,00:13
7,2.134218,2.184528,0.682073,00:13
8,2.092878,2.062588,0.670868,00:13
9,2.004410,2.103922,0.668067,00:13


In [6]:
learn.save('head-30-epochs')

Train for 10 more epochs.

In [7]:
learn.fit_one_cycle(10)

epoch,train_loss,valid_loss,error_rate,time
0,0.868251,2.042360,0.584034,00:13
1,0.930235,2.128511,0.591036,00:13
2,1.103251,2.167985,0.598039,00:13
3,1.172009,2.213692,0.579832,00:13
4,1.136868,2.183099,0.592437,00:13
5,1.030852,2.171098,0.584034,00:13
6,0.969445,2.158350,0.578431,00:13
7,0.876862,2.178020,0.577031,00:13
8,0.832732,2.178352,0.574230,00:13
9,0.821819,2.178206,0.574230,00:13


In [ ]:
learn.save('head-35-epochs')

Train for 5 more epochs.

In [ ]:
learn.fit_one_cycle(5)

In [ ]:
learn.save('head-15-epochs')

Train for 5 more epochs.

In [ ]:
learn.fit_one_cycle(5)

In [ ]:
learn.save('head-20-epochs')

Train for 5 more epochs.

In [ ]:
learn.fit_one_cycle(5)

In [ ]:
learn.save('head-25-epochs')

Train for 5 more epochs.

In [ ]:
learn.load('head-30-epochs');

Unfreeze entire model to train.

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

No lr seems to decrease the loss. Nevertheless try with a lr slice before the loss increases.

Train for 5 more epochs.

In [ ]:
learn.fit_one_cycle(1, max_lr=slice(1e-6,2e-6))

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.freeze()

In [ ]:
learn.fit_one_cycle(1, max_lr=slice(1e-6,1e-4))

In [ ]:
learn.save('body-38-epochs')

Train for 5 more epochs.

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, max_lr=slice(1e-6,5e-4))

In [ ]:
learn.save('body-30-epochs')

Train for 10 more epochs.

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(10, max_lr=slice(1e-6,5e-6))

In [ ]:
learn.save('body-40-epochs')

Training doesn't drastically improve the error rate. Even though training and validation loss are still decreasing, gains are marginal. 

## Interpretation

In [ ]:
learn.load('body-40-epochs');

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix(figsize=(8,8))

In [ ]:
interp.most_confused(min_val=2)

In [ ]:
interp.plot_top_losses(9, heatmap=True, figsize=(12,12))

Lots of mistakes are made by the model. Cleaning up the data might help as the scraped images contain noise or simply irrelevant images.

## Cleaning Up

In [ ]:
from fastai.widgets import *

First we need to get the file paths from our top_losses. We can do this with `.from_toplosses`. We then feed the top losses indexes and corresponding dataset to `ImageCleaner`.

Notice that the widget will not delete images directly from disk but it will create a new csv file `cleaned.csv` from where you can create a new ImageDataBunch with the corrected labels to continue training your model.

In order to clean the entire set of images, we need to create a new dataset without the split. The video lecture demostrated the use of the `ds_type` param which no longer has any effect. See [the thread](https://forums.fast.ai/t/duplicate-widget/30975/10) for more details.

In [ ]:
db = (ImageList.from_folder(path)
                   .split_none()
                   .label_from_folder()
                   .transform(get_transforms(), size=224)
                   .databunch()
     )

In [ ]:
# If you already cleaned your data using indexes from `from_toplosses`,
# run this cell instead of the one before to proceed with removing duplicates.
# Otherwise all the results of the previous step would be overwritten by
# the new run of `ImageCleaner`.

# db = (ImageList.from_csv(path, 'cleaned.csv', folder='.')
#                    .split_none()
#                    .label_from_df()
#                    .transform(get_transforms(), size=224)
#                    .databunch()
#      )

Then we create a new learner to use our new databunch with all the images.

In [ ]:
learn_cln = cnn_learner(db, models.resnet50, metrics=error_rate)

learn_cln.load('body-38-epochs');

In [ ]:
ds, idxs = DatasetFormatter().from_toplosses(learn_cln)

In [ ]:
losses,idxs = interp.top_losses(k=200)

In [ ]:
#! sudo chmod -R 777 /home/jupyter/tutorials/fastai/course-v3/nbs/dl1/data/mountains

In [ ]:
losses[:10]

In [ ]:
ImageCleaner(ds, idxs, path)

In [ ]:
ds, idxs = DatasetFormatter().from_similars(learn_cln)

In [ ]:
ImageCleaner(ds, idxs, path, duplicates=True)

Remember to recreate your ImageDataBunch from your `cleaned.csv` to include the changes you made in your data!

## Clean data

In [ ]:
df = pd.read_csv(path + '/cleaned.csv')
data_cln = ImageDataBunch.from_df(path, df=df, ds_tfms=get_transforms(), size=112, bs=32).normalize(imagenet_stats)

In [ ]:
data_cln.show_batch(rows=3, figsize=(7,8))

In [ ]:
learn_cln = cnn_learner(data_cln, models.resnet50, metrics=error_rate)

In [ ]:
learn_cln.load('body-40-epochs');

In [ ]:
learn_cln.lr_find()

In [ ]:
learn_cln.recorder.plot()

In [ ]:
learn_cln.fit_one_cycle(1, max_lr=slice(3e-4,1e-3))

In [ ]:
learn_cln.fit_one_cycle(1, max_lr=slice(1e-6,1e-4))

In [ ]:
learn_cln.unfreeze()

In [ ]:
learn_cln.lr_find()

In [ ]:
learn_cln.recorder.plot()

In [ ]:
learn_cln.fit_one_cycle(1, max_lr=slice(2e-5,3e-4))

## Interpretation

In [ ]:
learn.load('stage-2');

In [ ]:
interp = ClassificationInterpretation.from_learner(learn_cln)

In [ ]:
interp.plot_confusion_matrix(figsize=(8,8))

In [ ]:
interp.most_confused(min_val=2)

In [ ]:
interp.plot_top_losses(9, heatmap=True, figsize=(12,12))

## Training only with cleaned data

In [ ]:
224/2

In [ ]:
224*2

In [ ]:
df = pd.read_csv(path + '/cleaned.csv')
data_cln = ImageDataBunch.from_df(path, df=df, ds_tfms=get_transforms(), size=448, bs=32).normalize(imagenet_stats)

In [ ]:
data_cln.show_batch(rows=3, figsize=(7,8))

In [ ]:
learn_cln = cnn_learner(data_cln, models.resnet50, metrics=error_rate)

In [ ]:
learn_cln.load("test_resizing");

Train head for 5 epochs.

In [ ]:
learn_cln.fit_one_cycle(1)

In [ ]:
learn_cln.save("best_model_yet")

In [ ]:
learn_cln.recorder.plot_losses()

In [ ]:
learn_cln.save('cln_head-5-epochs')

Train for 5 more epochs.

In [ ]:
learn_cln.fit_one_cycle(1)

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.save('head-10-epochs')

Train for 5 more epochs.

In [ ]:
learn_cln.fit_one_cycle(5)

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.save('head-15-epochs')

Train for 5 more epochs.

In [ ]:
learn_cln.fit_one_cycle(5)

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.save('head-20-epochs')

Train for 5 more epochs.

In [ ]:
learn_cln.fit_one_cycle(5)

In [ ]:
learn.recorder.plot_losses()

Validation loss increases after epoch 15. Keep model at epoch 15.

In [ ]:
learn.load('head-15-epochs');

Unfreeze entire model to train.

In [ ]:
learn_cln.unfreeze()

In [ ]:
learn_cln.lr_find()

In [ ]:
learn_cln.recorder.plot()

No lr seems to decrease the loss. Nevertheless try with a lr slice before the loss increases.

Train for 5 more epochs.

In [ ]:
learn_cln.fit_one_cycle(1, max_lr=slice(5e-6,3e-5))

In [ ]:
learn_cln.lr_find()

In [ ]:
learn_cln.recorder.plot()

In [ ]:
learn_cln.freeze()

In [ ]:
learn_cln.save('test_resizing')

In [ ]:
learn_cln.fit_one_cycle(5, max_lr=slice(1e-6,1e-4))

In [ ]:
learn.save('body-25-epochs')

Train for 5 more epochs.

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, max_lr=slice(1e-6,5e-4))

In [ ]:
learn.save('body-30-epochs')

Train for 10 more epochs.

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(10, max_lr=slice(1e-6,5e-6))

In [ ]:
learn.save('body-40-epochs')

## Putting your model in production

First thing first, let's export the content of our `Learner` object for production:

In [ ]:
learn.export()

This will create a file named 'export.pkl' in the directory where we were working that contains everything we need to deploy our model (the model, the weights but also some metadata like the classes or the transforms/normalization used).

You probably want to use CPU for inference, except at massive scale (and you almost certainly don't need to train in real-time). If you don't have a GPU that happens automatically. You can test your model on CPU like so:

In [ ]:
defaults.device = torch.device('cpu')
print(defaults.device)

In [ ]:
img = open_image(path/'piz_bernina'/'00000008.jpg')
img

We create our `Learner` in production enviromnent like this, just make sure that `path` contains the file 'export.pkl' from before.

In [ ]:
learn = load_learner(path)

In [ ]:
pred_class,pred_idx,outputs = learn.predict(img)
pred_class.obj

So you might create a route something like this ([thanks](https://github.com/simonw/cougar-or-not) to Simon Willison for the structure of this code):

```python
@app.route("/classify-url", methods=["GET"])
async def classify_url(request):
    bytes = await get_bytes(request.query_params["url"])
    img = open_image(BytesIO(bytes))
    _,_,losses = learner.predict(img)
    return JSONResponse({
        "predictions": sorted(
            zip(cat_learner.data.classes, map(float, losses)),
            key=lambda p: p[1],
            reverse=True
        )
    })
```

(This example is for the [Starlette](https://www.starlette.io/) web app toolkit.)